In [1]:
import pydap.client
import numpy as np
import pandas as pd
import datetime
End_Year = 2019
Start_Year = 2017

In [2]:
url_dict = {}
url_dict['Precip']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
url_dict['Rhumiditymax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_'
url_dict['Rhumiditymin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmin/rmin_'
url_dict['Shumidit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/sph/sph_'
url_dict['Sradiation']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/srad/srad_'
url_dict['WindFromDir']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/th/th_'
url_dict['WindVelocity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vs/vs_'
url_dict['Tempmax']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmx/tmmx_'
url_dict['Tempmin']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmn/tmmn_'
url_dict['BurningIndex']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_'
url_dict['FuelMoist100']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm100/fm100_'
url_dict['FuelMoist1000']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/fm1000/fm1000_'
url_dict['EnergyRelease']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/erc/erc_'
#url_dict['DroughtSeverity']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pdsi/pdsi_'
url_dict['EvapoTranspiration']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/etr/etr_'
url_dict['EvapoTranspir_G']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pet/pet_'
url_dict['VaporPressDeficit']='http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vpd/vpd_'

In [3]:
base_url_part = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_'
url = base_url_part + str(End_Year) + '.nc'
dataset = pydap.client.open_url(url)

In [4]:
#Slice the Bounding Box we wanted here.
lon_ = dataset['lon']
lon = lon_[:].data
lat_ = dataset['lat']
lat = lat_[:].data

import numpy as np
SD_county_lats = np.where((lat<33.6) & (lat>32.4))
SD_county_lons = np.where((lon<-116.0) & (lon>-118))

count_of_lats = lat[SD_county_lats].shape[0]
count_of_lons = lon[SD_county_lons].shape[0]

In [7]:
flag=0
for Year in range(End_Year,Start_Year-1,-1):
    url = base_url_part + str(Year) + '.nc'
    dataset = pydap.client.open_url(url)
    #Create the Base DataFrame with Days and Co-ordinates
    Days_in_year = dataset['day'].shape[0]
    index = range(Days_in_year*count_of_lats*count_of_lons)
    columns = ['Date','coord']
    base_df = pd.DataFrame(index=index,columns=columns) 
    idx=0
    for day in range(Days_in_year):
        date = datetime.date(Year, 1, 1) + datetime.timedelta(day)
        for i in range(count_of_lats):
            for j in range(count_of_lons):
                base_df.loc[idx]['Date'] = date
                base_df.loc[idx]['coord'] = (lat[SD_county_lats][i],lon[SD_county_lons][j])
                idx = idx+1
    ##DataFrame base Created
    display (base_df.head())
    print ("Base DataFrame created of size ", base_df.shape)
    ##Load each Attriubute below
    for item in url_dict.values():
        print ("Processing for", item)
        url = item + str(Year) + '.nc'
        print (url)
        dataset = pydap.client.open_url(url)
        print (dataset.keys())
        for i in dataset.keys():
            if ((i == 'lat') or (i == 'lon') or (i == 'crs') or (i == 'day')):
                pass
            else:
                column = i
        print (column)
        precip = dataset[column]
        #Here is where the Data Download happens
        key_data = precip[:].data
        print ("Data downloaded")
        values = []
        for i in range(key_data[0].shape[0]):
            sam = key_data[0][i][SD_county_lats[0][0]:SD_county_lats[0][-1]+1,SD_county_lons[0][0]:SD_county_lons[0][-1]+1].flatten().tolist()
            values.extend(sam)
        if not column in base_df.columns:
            base_df[column]=values
        else:
            temp = column + '_again'
            base_df[temp]=values
        display(base_df.head())
    print ("DataFrame for the Year", Year , "created of size ", base_df.shape)
    ##All Attributes loaded for a given year
    ##Merge the DF with Previous Year
    if flag==0:
        flag = 1
        final_df = base_df
    else:
        final_df = pd.concat([final_df,base_df],ignore_index=True)
        print ("DataFrame size after adding for year", Year ,"is" ,final_df.shape)

,Date,coord
0,2019-01-01,"(33.56666666666667, -117.97499996666667)"
1,2019-01-01,"(33.56666666666667, -117.93333330000002)"
2,2019-01-01,"(33.56666666666667, -117.89166663333334)"
3,2019-01-01,"(33.56666666666667, -117.84999996666667)"
4,2019-01-01,"(33.56666666666667, -117.80833330000002)"


Base DataFrame created of size  (508080, 2)
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pr/pr_2019.nc
KeysView(<DatasetType with children 'precipitation_amount', 'lat', 'day', 'crs', 'lon'>)
precipitation_amount
Data downloaded


,Date,coord,precipitation_amount
0,2019-01-01,"(33.56666666666667, -117.97499996666667)",0
1,2019-01-01,"(33.56666666666667, -117.93333330000002)",0
2,2019-01-01,"(33.56666666666667, -117.89166663333334)",0
3,2019-01-01,"(33.56666666666667, -117.84999996666667)",0
4,2019-01-01,"(33.56666666666667, -117.80833330000002)",0


DataFrame for the Year 2019 created of size  (508080, 3)
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_2019.nc
KeysView(<DatasetType with children 'relative_humidity', 'lat', 'day', 'crs', 'lon'>)
relative_humidity
Data downloaded


,Date,coord,precipitation_amount,relative_humidity
0,2019-01-01,"(33.56666666666667, -117.97499996666667)",0,496
1,2019-01-01,"(33.56666666666667, -117.93333330000002)",0,478
2,2019-01-01,"(33.56666666666667, -117.89166663333334)",0,470
3,2019-01-01,"(33.56666666666667, -117.84999996666667)",0,472
4,2019-01-01,"(33.56666666666667, -117.80833330000002)",0,478


...

DataFrame for the Year 2017 created of size  (508080, 16)
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pet/pet_
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/pet/pet_2017.nc
KeysView(<DatasetType with children 'lat', 'lon', 'potential_evapotranspiration', 'crs', 'day'>)
potential_evapotranspiration
Data downloaded


,Date,coord,precipitation_amount,relative_humidity,relative_humidity_again,specific_humidity,surface_downwelling_shortwave_flux_in_air,wind_from_direction,wind_speed,air_temperature,air_temperature_again,burning_index_g,dead_fuel_moisture_100hr,dead_fuel_moisture_1000hr,energy_release_component-g,potential_evapotranspiration,potential_evapotranspiration_again
0,2017-01-01,"(33.56666666666667, -117.97499996666667)",0,821,494,510,1231,256,34,668,710,25,181,191,24,27,18
1,2017-01-01,"(33.56666666666667, -117.93333330000002)",0,803,504,511,1196,256,34,666,713,25,181,193,23,26,18
2,2017-01-01,"(33.56666666666667, -117.89166663333334)",0,810,500,507,1161,256,33,666,710,26,177,188,25,26,18
3,2017-01-01,"(33.56666666666667, -117.84999996666667)",0,862,498,503,1125,248,33,665,696,25,179,187,24,24,16
4,2017-01-01,"(33.56666666666667, -117.80833330000002)",0,881,469,493,1090,248,33,670,688,27,174,177,27,25,17


DataFrame for the Year 2017 created of size  (508080, 17)
Processing for http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vpd/vpd_
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/vpd/vpd_2017.nc
KeysView(<DatasetType with children 'lat', 'lon', 'mean_vapor_pressure_deficit', 'crs', 'day'>)
mean_vapor_pressure_deficit
Data downloaded


,Date,coord,precipitation_amount,relative_humidity,relative_humidity_again,specific_humidity,surface_downwelling_shortwave_flux_in_air,wind_from_direction,wind_speed,air_temperature,air_temperature_again,burning_index_g,dead_fuel_moisture_100hr,dead_fuel_moisture_1000hr,energy_release_component-g,potential_evapotranspiration,potential_evapotranspiration_again,mean_vapor_pressure_deficit
0,2017-01-01,"(33.56666666666667, -117.97499996666667)",0,821,494,510,1231,256,34,668,710,25,181,191,24,27,18,48
1,2017-01-01,"(33.56666666666667, -117.93333330000002)",0,803,504,511,1196,256,34,666,713,25,181,193,23,26,18,48
2,2017-01-01,"(33.56666666666667, -117.89166663333334)",0,810,500,507,1161,256,33,666,710,26,177,188,25,26,18,48
3,2017-01-01,"(33.56666666666667, -117.84999996666667)",0,862,498,503,1125,248,33,665,696,25,179,187,24,24,16,43
4,2017-01-01,"(33.56666666666667, -117.80833330000002)",0,881,469,493,1090,248,33,670,688,27,174,177,27,25,17,45


DataFrame for the Year 2017 created of size  (508080, 18)
DataFrame size after adding for year 2017 is (1524240, 18)
